In [ ]:
import json
import torch
import spacy
import common
from transformers import AutoModelForTokenClassification, AutoTokenizer, pipeline


In [ ]:
sub_cirs = range(21916, 21916+50)
    #[21916, 21917,21923, 21924, 21930,21941,22115,22281,22942]
#sub_cirs = [21917]
print([c for c in sub_cirs])


In [ ]:
output = common.get_circulars(sub_cirs)
print(output[0])

In [ ]:
remote_model_path = 'kusha7/astrobert-gcn-tokenizer'
hf_token=os.environ["hf_token"]
tokenizer = AutoTokenizer.from_pretrained(remote_model_path, token=hf_token)
model = AutoModelForTokenClassification.from_pretrained(
    "kusha7/astrobert-gcn-tokenizer", token=hf_token
)
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="none")

In [ ]:
nlp = spacy.load("en_core_web_sm")
label_list = model.config.id2label
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
def get_att(cir):
    text = cir["body"]
    doc = nlp(text)
    sentences = [sent.text.strip() for sent in doc.sents]
    # print(sentences)
    full_tokens = []
    for sent in sentences:
        print("sent", sent)
        inputs = tokenizer(sent, return_tensors="pt", return_offsets_mapping=True, truncation=True)
        offset_mapping = inputs.pop("offset_mapping")
        # Move input tensors to device
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs, return_dict=True)

        logits = outputs.logits.cpu()

        predictions = torch.argmax(logits, dim=2)

        tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
        offsets = offset_mapping[0].tolist()

        # Format output

        for token, offset, pred_id in zip(tokens, offsets, predictions[0]):
            label = label_list[pred_id.item()]
            full_tokens.append({
                "token": token,
                "start": offset[0],
                "end": offset[1],
                "label": label
            })

        # Output includes all tokens — even those labeled "O"
    return common.merge_subwords(full_tokens)

In [ ]:
def build_atts_from_cir(json_circulars):
    new_list = []
    for sub_cir in json_circulars:
        new_obj = sub_cir.copy()
        text = sub_cir["body"]
        new_obj["body"] = text.replace("\n", " ")
        new_obj["new_pipeline_output"] = get_att(new_obj)
        new_list.append(new_obj)
    return new_list


In [ ]:
output_bio_2 = build_atts_from_cir(output)
# print(output_bio_2[0]["body"])
print(output_bio_2[0]["new_pipeline_output"])
